# Elektrolumineszenz-Prüfung von Solarzellen - klassische Bildverarbeitung

Programmgerüst als Hilfestellung zu Labor 2

Bitte zunächst Laboraufgabe und dann die per `# TODO` gekennzeichneten Hinweise beachten

## 1 Anzeigen der Bilder


In [ ]:
import cv2
import os
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pprint

# Pfadangaben sind an den eigenen Speicherort der Datei splitted.zip anzupassen
BASE_PATH = "../data"
TRAIN_PATH = "train"
VALIDATION_PATH = "val"

SHOW = False  # ermöglicht eine Kurzansicht der Bilddateien, wenn auf True gesetzt
EXPORT_REPORT = False  # exportiere Zwischenschritte und Endergebnisse in Datei

if EXPORT_REPORT:
    # Matplotlib plot Einstellungen für Laborbericht setzen
    rc_fonts = {
        "font.family": "serif",
        "text.usetex": True,
        "text.latex.preamble": r"\usepackage{libertine}",
    }
    mpl.rcParams.update(rc_fonts)

# Einlesen aller Verzeichnisse
train_path = os.path.join(BASE_PATH, TRAIN_PATH)
defect_classes = os.listdir(train_path)
print("[i] Fehlerklassen in %s: %s" % (train_path, defect_classes))

# Anzeige aller Bilder der einzelnen Verzeichnisse, falls SHOW auf True gesetzt
for class_idx, defect_class in enumerate(defect_classes):
    files = os.listdir(os.path.join(train_path, defect_class))
    print("[i] Klasse", defect_class, "mit %d Bildern" % (len(files)))

    if SHOW:
        for file_idx, file in enumerate(files):
            img = cv2.imread(os.path.join(train_path, defect_class, file))
            cv2.imshow("Original", img)
            cv2.waitKey(100)
    cv2.destroyAllWindows()

## 2 Anzeigen des jeweils ersten Bildes einer Fehlerklasse als Grundlage für die Entwicklung


In [ ]:
# Anzeige des ersten Bildes je Verzeichnis
plt.figure(figsize=(15, 15))

for class_idx, defect_class in enumerate(defect_classes):
    files = os.listdir(os.path.join(train_path, defect_class))

    img_bgr = cv2.imread(os.path.join(train_path, defect_class, files[0]))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    plt.subplot(1, len(defect_classes), class_idx + 1)
    plt.imshow(img_rgb)
    plt.title(defect_class)

## 3 Entwicklung der Segmentierung `my_segmentation` und Merkmalsextraktion `my_feature_extraction`

**Hinweis**: Beschränken Sie sich zunächst auf einfache Filter wie Glättung und Median-Filter sowie die Auswahl von Bereichen auf der Basis der Helligkeit auch unter Nutzung der Funktion `cv2.adaptiveThreshold` und kombinieren Sie ggfs. mehrere Segmentierungsansätze zu einem Binärbild.

Um eine flexible Merkmalsextraktion zu ermöglichen, übergeben Sie die Merkmale als Dictionary, z.B.:

```
features = {
    'area': 1000,
    'perimeter' : 345,
    'circularity' : 0.5,
    ...
}


print(features["area"])
```


### Allgemeine Segmentierungsfunktionen

Sammlung von Funktionen zum vor-verarbeiten eines Grauwertbildes.


In [ ]:
def apply_shading_correction(img: np.ndarray, brightness: float) -> np.ndarray:
    X, Y = np.meshgrid(
        np.linspace(-1, 1, img.shape[0]), np.linspace(-1, 1, img.shape[1])
    )
    img_shading = (
        img
        + np.exp(
            -(
                (1.5 * (X / np.abs(X).max() - 1)) ** 2 / 2
                + (2 * (Y / np.abs(X).max() + 1)) ** 2 / 2
            )
        )
        * img.max()
        * brightness
    )
    return ((img_shading / img_shading.max()) * 255).astype(np.uint8)


def create_bus_mask(img_canny: np.ndarray, fig: plt.Figure = None) -> np.ndarray:
    edge_finder = np.hstack(
        (
            np.full((img_canny.shape[0], 15), img_canny.min()),
            np.full((img_canny.shape[0], 15), img_canny.max()),
            np.full((img_canny.shape[0], 15), img_canny.min()),
        )
    )

    x_corr = cv2.matchTemplate(img_canny, edge_finder, cv2.TM_CCORR)[0, :]
    x_peaks = np.array(
        [
            np.argmax(x_corr[: x_corr.size // 2]),
            np.argmax(x_corr[x_corr.size // 2 :]) + x_corr.size // 2,
        ]
    )
    x_edges = x_peaks + edge_finder.shape[1] // 2

    y_corr = cv2.matchTemplate(img_canny, edge_finder.T, cv2.TM_CCORR)[:, 0]
    y_peaks = np.array(
        [
            np.argmax(y_corr[: y_corr.size // 16]),
            np.argmax(y_corr[15 * y_corr.size // 16 :]) + 15 * y_corr.size // 16,
        ]
    )
    y_edges = y_peaks + edge_finder.shape[1] // 2
    y_bar_peaks = np.array(
        [
            np.argmax(y_corr[y_corr.size // 16 : y_peaks[0] + 140]) + y_corr.size // 16,
            np.argmax(y_corr[y_peaks[0] + 140 : y_corr.size // 4]) + y_peaks[0] + 140,
            np.argmax(y_corr[y_corr.size // 4 : y_corr.size // 2]) + y_corr.size // 4,
            np.argmax(y_corr[y_corr.size // 2 : 3 * y_corr.size // 4])
            + y_corr.size // 2,
            np.argmax(y_corr[3 * y_corr.size // 4 : y_peaks[1] - 140])
            + 3 * y_corr.size // 4,
            np.argmax(y_corr[y_peaks[1] - 140 : 15 * y_corr.size // 16])
            + y_peaks[1]
            - 140,
        ]
    )
    y_bars = (
        y_bar_peaks.reshape(-1, 2).mean(axis=1).astype("int")
        + edge_finder.shape[1] // 2
    )

    img_mask = np.ones_like(img_canny, dtype=img_canny.dtype)
    img_mask[:, : x_edges[0]] = np.zeros((img_mask.shape[0], x_edges[0]))
    img_mask[:, x_edges[1] :] = np.zeros(
        (img_mask.shape[0], img_mask.shape[1] - x_edges[1])
    )
    bar_half_width = 25
    for bar in y_bars:
        y_corr[bar - bar_half_width : bar + bar_half_width] = np.zeros(
            (bar_half_width * 2,)
        )
        img_mask[bar - bar_half_width : bar + bar_half_width, :] = np.zeros(
            (bar_half_width * 2, img_mask.shape[1])
        )

    img_mask[: y_edges[0], :] = np.zeros((y_edges[0], img_mask.shape[1]))
    img_mask[y_edges[1] :, :] = np.zeros(
        (img_mask.shape[0] - y_edges[1], img_mask.shape[1])
    )
    corner_size = 70
    for y_idx, y in enumerate(np.arange(y_edges[0], y_edges[0] + corner_size)):
        for x in np.arange(x_edges[0], x_edges[0] + corner_size):
            img_mask[y, max(x - y_idx, x_edges[0])] = 0
    for y_idx, y in enumerate(np.arange(y_edges[0], y_edges[0] + corner_size)):
        for x in np.arange(x_edges[1] - corner_size, x_edges[1]):
            img_mask[y, min(x + y_idx, x_edges[1])] = 0
    for y_idx, y in enumerate(np.arange(y_edges[1] - corner_size, y_edges[1])):
        for x in np.arange(x_edges[0], x_edges[0] + corner_size):
            img_mask[y, max(x - (corner_size - y_idx), x_edges[0])] = 0
    for y_idx, y in enumerate(np.arange(y_edges[1] - corner_size, y_edges[1])):
        for x in np.arange(x_edges[1] - corner_size, x_edges[1]):
            img_mask[y, min(x + (corner_size - y_idx), x_edges[1])] = 0

    if fig is not None:
        ax_img = fig.add_axes([0.3, 0.3, 0.6, 0.6])
        ax_finder_y = fig.add_axes([0.3, 0.9, 0.6, 0.05])
        ax_corr_y = fig.add_axes([0.05, 0.3, 0.225, 0.6])
        ax_finder_x = fig.add_axes([0.9, 0.3, 0.05, 0.6])
        ax_corr_x = fig.add_axes([0.3, 0.05, 0.6, 0.225])

        ax_finder_x.imshow(edge_finder, cmap="Greys")
        ax_finder_x.set_xticks([])
        ax_finder_x.set_yticks([])
        ax_finder_x.get_yaxis().set_label_position("right")
        ax_finder_x.set_ylabel("X Edge-Finder (invertiert)")
        ax_finder_y.imshow(edge_finder.T, cmap="Greys")
        ax_finder_y.set_xticks([])
        ax_finder_y.set_yticks([])
        ax_finder_y.get_xaxis().set_label_position("top")
        ax_finder_y.set_xlabel("Y Edge-Finder (invertiert)")

        y_corr_norm = np.pad(
            y_corr / y_corr.max(),
            (
                int(np.floor(edge_finder.shape[1] / 2)),
                int(np.ceil(np.floor(edge_finder.shape[1] / 2))),
            ),
        )
        ax_corr_y.plot(y_corr_norm, np.arange(len(y_corr_norm)))
        ax_corr_y.set_xlabel("Y Korrelation [norm]")
        ax_corr_y.set_xlim([0, 1])
        ax_corr_y.set_ylim([0, len(y_corr_norm)])
        ax_corr_y.grid(True)
        ax_corr_y.get_xaxis().tick_top()
        ax_corr_y.get_xaxis().set_label_position("top")
        ax_corr_y.set_xticks(np.linspace(0, 1, 5, endpoint=True))

        ax_img.imshow(img_canny, cmap="Greys")
        ax_img.set_xticks([])
        ax_img.set_yticks([])

        x_corr_norm = np.pad(
            x_corr / x_corr.max(),
            (
                int(np.floor(edge_finder.shape[1] / 2)),
                int(np.ceil(np.floor(edge_finder.shape[1] / 2))),
            ),
        )
        ax_corr_x.plot(np.arange(len(x_corr_norm)), x_corr_norm)
        ax_corr_x.set_ylabel("X Korrelation [norm]")
        ax_corr_x.set_xlim([0, len(y_corr_norm)])
        ax_corr_x.set_ylim([0, 1])
        ax_corr_x.grid(True)
        ax_corr_x.set_yticks(np.linspace(0, 1, 5, endpoint=True))

    return img_mask


def find_canny_edges(img: np.ndarray) -> np.ndarray:
    img_med_blurred = cv2.medianBlur(img, ksize=11)
    return cv2.Canny(img_med_blurred, np.percentile(img, 7), np.percentile(img, 5))

### Segmentierung: a)


In [ ]:
def find_defects_a(img: np.ndarray, img_mask: np.ndarray) -> np.ndarray:
    _, img_on = cv2.threshold(img, np.percentile(img, 60), 1, cv2.THRESH_BINARY)
    _, img_off = cv2.threshold(img, np.percentile(img, 10), 1, cv2.THRESH_BINARY)

    img_panel = cv2.adaptiveThreshold(
        img,
        255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV,
        101,
        5,
    )
    img_panel = (img_panel & ~img_on & img_mask) | (img_mask & ~img_off)
    s = np.array([1, 0, 1], ndmin=2, dtype=img_panel.dtype)
    img_defects = cv2.erode(img_panel, s, iterations=1)
    return img_defects


def apply_segmentation_a(img: np.ndarray, dbg_fig: plt.Figure = None) -> np.ndarray:
    """Hier entwickeln Sie die Bildvorverarbeitung und Segmentierung

    Parameter img: einkanaliges Grauwertbild vom Typ np.uint8
    Rückgabe img_final: einkanaliges Binärbild mit Wert 0 oder 255 vom Typ np.uint8

    Beschreibung:
    Diese Funktion beinhaltet Vorverarbeitung und Segmentierung und soll
    möglichst alle Fehler idealerweise vollständig als Binärbild segmentieren
    und dabei kleine Artefakte eliminieren.

    """
    img_overexposed = ((img / np.percentile(img, 70)).clip(0, 1) * 255).astype(np.uint8)

    img_shading = apply_shading_correction(img_overexposed, 0.08)

    img_canny = find_canny_edges(img_shading)

    img_mask = create_bus_mask(img_canny, fig=dbg_fig)

    img_final = find_defects_a(img_shading, img_mask)

    return ((img_final / img_final.max()) * 255).astype(np.uint8)

### Merkmalsextraktion: a)


In [ ]:
def exctact_features_a(img_bin: np.ndarray):
    """Hier führen Sie die Merkmalsextration aus und visualisieren das Ergebnis

    Parameter img_bin: segmentiertes Binärbild mit den Werten 0 oder 255 vom Datentyp np.uint8
    Rückgabe cnt: Kontur des flächenmässig groessten Segments
    Rückgabe features: Dictionary, das die Merkmale des durch cnt beschriebenen Segments zurückgibt

    Diese Funktion ermittelt die von Ihnen ausgewählten Merkmale und speichert diese in einem Dictionary.
    Es bietet sich an, hier mehr als 4 Merkmale zu berechnen, so dass spaeter die bestgeeignete Kombination
    ausgewählt werden kann.

    Hierbei ist zu beachten, dass auch eine leere Kontur behandelt werden muss und ein Dictionary zurückliefert.
    In diesem Fall kann eine leere Kontur zurückgegeben werden, z.B. per cnt = None.

    """

    cnts, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    top_cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:1]
    moments = [cv2.moments(cnt) for cnt in top_cnts]
    directions = [
        0.5 * np.arctan2(2 * m["mu11"], m["mu20"] - m["mu02"]) for m in moments
    ]
    perimeters = [cv2.arcLength(cnt, True) for cnt in top_cnts]
    eccentricity = [
        ((m["mu20"] - m["mu02"]) ** 2 + 4 * m["mu11"] ** 2)
        / (m["mu20"] + m["mu02"]) ** 2
        for m in moments
    ]

    cnt = max(cnts, key=cv2.contourArea)
    features = {
        "area": [i for i in map(cv2.contourArea, top_cnts)],
        "direction": directions,
        "perimeter": perimeters,
        "eccentricity": eccentricity,
    }

    return cnt, features

### Segmentierung: b)


In [ ]:
def find_defects_b(img: np.ndarray, img_mask: np.ndarray) -> np.ndarray:
    # panel extraction
    img_panel_adapt_thresh = cv2.adaptiveThreshold(
        cv2.medianBlur(img, ksize=1),
        255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        201,
        3,
    )
    img_panel_gauss_blurred = cv2.GaussianBlur(
        img_panel_adapt_thresh, ksize=(3, 1), sigmaX=0
    )
    img_panel_blurred = cv2.medianBlur(img_panel_gauss_blurred, ksize=3)
    _, img_panel_thresh = cv2.threshold(
        img_panel_blurred, 30, 255, cv2.THRESH_BINARY_INV
    )

    # masking
    img_panel_thresh &= img_mask

    # cell border suppression
    s_panel_dilate = cv2.getStructuringElement(cv2.MORPH_CROSS, (7, 11))
    img_panel_dilate = cv2.dilate(img_panel_thresh, s_panel_dilate, iterations=2)
    s_panel_erode = cv2.getStructuringElement(cv2.MORPH_CROSS, (11, 7))
    img_panel_erode = cv2.erode(img_panel_dilate, s_panel_erode, iterations=1)

    return img_panel_erode


def apply_segmentation_b(img: np.ndarray, dbg_fig: plt.Figure = None) -> np.ndarray:
    """Hier entwickeln Sie die Bildvorverarbeitung und Segmentierung

    Parameter img: einkanaliges Grauwertbild vom Typ np.uint8
    Rückgabe img_final: einkanaliges Binärbild mit Wert 0 oder 255 vom Typ np.uint8

    Beschreibung:
    Diese Funktion beinhaltet Vorverarbeitung und Segmentierung und soll
    möglichst alle Fehler idealerweise vollständig als Binärbild segmentieren
    und dabei kleine Artefakte eliminieren.

    """
    img_overexposed = ((img / np.percentile(img, 70)).clip(0, 1) * 255).astype(np.uint8)

    img_shading = apply_shading_correction(img_overexposed, 0.1)

    img_canny = find_canny_edges(img_shading)

    img_mask = create_bus_mask(img_canny, fig=dbg_fig)

    img_final = find_defects_b(img_shading, img_mask)

    return ((img_final / img_final.max()) * 255).astype(np.uint8)

### Merkmalsextraktion: b)


In [ ]:
def exctact_features_b(img_bin: np.ndarray):
    """Hier führen Sie die Merkmalsextration aus und visualisieren das Ergebnis

    Parameter img_bin: segmentiertes Binärbild mit den Werten 0 oder 255 vom Datentyp np.uint8
    Rückgabe cnt: Kontur des flächenmässig groessten Segments
    Rückgabe features: Dictionary, das die Merkmale des durch cnt beschriebenen Segments zurückgibt

    Diese Funktion ermittelt die von Ihnen ausgewählten Merkmale und speichert diese in einem Dictionary.
    Es bietet sich an, hier mehr als 4 Merkmale zu berechnen, so dass spaeter die bestgeeignete Kombination
    ausgewählt werden kann.

    Hierbei ist zu beachten, dass auch eine leere Kontur behandelt werden muss und ein Dictionary zurückliefert.
    In diesem Fall kann eine leere Kontur zurückgegeben werden, z.B. per cnt = None.

    """

    cnts, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    top_cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:1]
    moments = [cv2.moments(cnt) for cnt in top_cnts]
    areas = [i for i in map(cv2.contourArea, top_cnts)]
    solidities = [
        area / cv2.contourArea(cv2.convexHull(cnt))
        for area, cnt in zip(areas, top_cnts)
    ]
    directions = [
        0.5 * np.arctan2(2 * m["mu11"], m["mu20"] - m["mu02"]) for m in moments
    ]
    eccentricity = [
        ((m["mu20"] - m["mu02"]) ** 2 + 4 * m["mu11"] ** 2)
        / (m["mu20"] + m["mu02"]) ** 2
        for m in moments
    ]

    cnt = max(cnts, key=cv2.contourArea)
    features = {
        "area": areas,
        "solidity": solidities,
        "direction": directions,
        "eccentricity": eccentricity,
    }

    return cnt, features

### Varianten

Hier werden die Segmentierungs- und Merkmalsextraktionsvarianten kombiniert.

* Variante 1:
  * Segmentierung: a)
  * Merkmalsextraktion: a)
* Variante 2:
  * Segmentierung: a)
  * Merkmalsextraktion: b)
* Variante 3:
  * Segmentierung: b)
  * Merkmalsextraktion: a)
* Variante 4:
  * Segmentierung: b)
  * Merkmalsextraktion: b)


In [ ]:
variants = [
    {
        "apply_segmentation": apply_segmentation_a,
        "exctact_features": exctact_features_a,
    },
    {
        "apply_segmentation": apply_segmentation_a,
        "exctact_features": exctact_features_b,
    },
    {
        "apply_segmentation": apply_segmentation_b,
        "exctact_features": exctact_features_a,
    },
    {
        "apply_segmentation": apply_segmentation_b,
        "exctact_features": exctact_features_b,
    },
]

### Parametrierung


In [ ]:
variant = 0  # wähle Variante der Segmentierung/Merkmalsextraktion
k = 3  # wähle Anzahl Nachbarin in k-nearest neighbor

# variant muss ein index innerhalb des variants arrays sein
assert variant >= 0 and variant < len(variants)

# k muss ungerade sein, um eine Mehrheitsentscheidung zu erlauben
assert k % 2 == 1

### Anwenden der Segmentierung und Merkmalsextraktion


In [ ]:
# Anzeige des ersten Bildes je Verzeichnis als Original, Segmentiertes Binärbild und als Bounding Box im Originalbild
fig = plt.figure(figsize=(30, 20), layout="tight")

class_count = len(defect_classes)

for class_idx, defect_class in enumerate(defect_classes):
    files = os.listdir(os.path.join(train_path, defect_class))

    img_bgr = cv2.imread(os.path.join(train_path, defect_class, files[0]))
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    ax = fig.add_subplot(3, class_count, class_idx + 1)
    ax.imshow(img_rgb)
    ax.set_title(defect_class)

    img_gray_filtered = variants[variant]["apply_segmentation"](img_gray)
    img_rgb_filtered = cv2.cvtColor(img_gray_filtered, cv2.COLOR_GRAY2RGB)
    ax = fig.add_subplot(3, class_count, class_idx + 1 + class_count)
    ax.imshow(img_rgb_filtered)
    ax.set_title("filtered " + defect_class)

    cnt, features = variants[variant]["exctact_features"](img_gray_filtered)

    x, y, w, h = cv2.boundingRect(cnt)
    img_rgb_cnt = cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (255, 0, 0), 5)
    ax = fig.add_subplot(3, class_count, class_idx + 1 + 2 * class_count)
    ax.imshow(img_rgb_cnt)
    ax.set_title("segmented " + defect_class)

    print(defect_class)
    pprint.pprint(features)

if EXPORT_REPORT:
    dbg_fig = plt.figure(figsize=(5.7, 5.7), layout="tight")
    img_gray_filtered = variants[variant]["apply_segmentation"](
        img_gray, dbg_fig=dbg_fig
    )
    dbg_fig.savefig(f"report/images/bus_mask_{defect_classes[-1]}.eps", pad_inches=0)

## 4 Merkmalsraum für alle Trainingsdaten visualisieren

Dieses Skript visualisiert den Merkmalsraum fuer genau 4 Merkmale und stellt jeweils die Kombination des ersten Merkmals mit jedem der anderen Merkmalen in einem zweidimensionalen Merkmalsraum dar.


In [ ]:
# Festlegung der auszuwaehlenden Features - Die Anzeige ist auf genau 4 Merkmale programmiert
FEATURES = [
    [
        "area",
        "direction",
        "perimeter",
        "eccentricity",
    ],
    [
        "area",
        "solidity",
        "direction",
        "eccentricity",
    ],
    [
        "area",
        "direction",
        "perimeter",
        "eccentricity",
    ],
    [
        "area",
        "solidity",
        "direction",
        "eccentricity",
    ],
]

# Iteriere ueber alle Bilder, deren Klassenzugehoerigkeit durch defect_class bestimmt wird
x_train = []
y_train = []

for class_idx, defect_class in enumerate(defect_classes):
    files = os.listdir(os.path.join(train_path, defect_class))
    print("[i] Klasse", defect_class, "mit %d Bildern" % (len(files)))

    for file_idx, file in enumerate(files):
        img_bgr = cv2.imread(os.path.join(train_path, defect_class, file))
        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        img_gray_filtered = variants[variant]["apply_segmentation"](img_gray)
        cnt, features = variants[variant]["exctact_features"](img_gray_filtered)
        if features:
            x_train.append(
                [
                    features[FEATURES[variant][0]],
                    features[FEATURES[variant][1]],
                    features[FEATURES[variant][2]],
                    features[FEATURES[variant][3]],
                ]
            )
            y_train.append(class_idx)

x_train = np.array(x_train)
y_train = np.array(y_train)

# Anzeige des Merkmalsraums
markers = ["or", "og", "ob", "ok"]
fig = plt.figure("Merkmalsraum", figsize=(10, 3), layout="tight")

for f, feature in enumerate(FEATURES[variant]):
    if f == 0:
        continue  # erste Grafik (FEATURES[variant][0] über sich selbst) wird übersprungen
    ax = fig.add_subplot(1, 3, f)
    for i, label in enumerate(defect_classes):
        ax.plot(
            x_train[y_train == i, 0],
            x_train[y_train == i, f],
            markers[i],
            label=f"\\texttt{{{label}}}",
            ms=2.5,
        )
    if f == len(FEATURES[variant]) // 2:
        ax.legend()
    ax.set_xlabel(FEATURES[variant][0])
    ax.set_ylabel(feature)
    ax.set_title(FEATURES[variant][0] + " vs. " + feature)

if EXPORT_REPORT:
    fig.savefig(f"report/images/feature_space_var{variant}.eps", pad_inches=0)

## 5 Training

**Wichtig**: Normierung ist nicht zu vergessen und bei der Klassifizierung müssen die hier erzeugten Normierungsparameter ohne Neuberechnung wiederverwendet werden.


In [ ]:
means = np.mean(x_train, axis=0)  # Spaltenweise Mittelwerte berechnen
stds = np.std(x_train, axis=0)  # Spaltenweise Standardabweichung berechnen

train = (x_train - means) / stds

## 6 Klassifizierung


In [ ]:
# Iteriere über alle Bilder, deren Klassenzugehörigkeit durch defect_class bestimmt wird
x_val = []
y_val = []

# Einlesen aller Verzeichnisse
validation_path = os.path.join(BASE_PATH, VALIDATION_PATH)
print(defect_classes == os.listdir(validation_path))

filenames = []

for class_idx, defect_class in enumerate(defect_classes):
    files = os.listdir(os.path.join(validation_path, defect_class))
    print("[i] Klasse", defect_class, "mit %d Bildern" % (len(files)))

    for file_idx, file in enumerate(files):
        filenames.append(os.path.join(validation_path, defect_class, file))
        img_bgr = cv2.imread(os.path.join(validation_path, defect_class, file))
        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        img_gray_filtered = variants[variant]["apply_segmentation"](img_gray)
        cnt, features = variants[variant]["exctact_features"](img_gray_filtered)

        x_val.append(
            [
                features[FEATURES[variant][0]],
                features[FEATURES[variant][1]],
                features[FEATURES[variant][2]],
                features[FEATURES[variant][3]],
            ]
        )
        y_val.append(class_idx)

x_val = np.array(x_val)
y_val = np.array(y_val).astype(np.float32)

In [ ]:
val = (x_val - means) / stds

y_val_predicted = np.zeros(val.shape[0])

for i in range(val.shape[0]):
    distances = np.linalg.norm(val[i] - train, axis=1)
    k_nearest_idx = np.argsort(distances.ravel())[:k]
    kinds, counts = np.unique(y_train[k_nearest_idx], return_counts=True)
    top_count = np.argmax(counts)
    y_val_predicted[i] = kinds[top_count]

print(f"Accuracy: {(y_val_predicted == y_val).mean() * 100:.2f}%")

## 7 Analyse der Ergebnisse ueber eine Confusion Matrix


In [ ]:
# ggfs. ist sklearn in Eingabekonsole nach Aufruf von "conda activate" über "pip install sklearn" zu installieren
from sklearn.metrics import confusion_matrix

fig = plt.figure(figsize=(3, 3), layout="tight")
ax = fig.add_subplot()

conf_mat = confusion_matrix(y_val_predicted.astype(int), y_val.astype(int))
ax.imshow(conf_mat, cmap="Blues")
indexes = np.arange(len(defect_classes))

for i in indexes:
    for j in indexes:
        ax.text(j, i, conf_mat[i, j])
ax.set_xticks(indexes, defect_classes, rotation=90)
ax.set_xlabel("Wahrheit")
ax.set_yticks(indexes, defect_classes)
ax.set_ylabel("Vorhersage")
ax.set_title("Confusion Matrix")

if EXPORT_REPORT:
    ax.set_title(None)
    fig.savefig(f"report/images/confusion_matrix_var{variant}.eps", pad_inches=0)

## 8 Detailanalyse der fehlerhaften Klassifizierungen mit Anzeige der Bilder


In [ ]:
for i in range(len(y_val)):
    class_predicted = defect_classes[int(y_val_predicted[i])]
    class_truth = defect_classes[int(y_val[i])]
    comment = (
        f"Vorhersage: \\texttt{{{class_predicted}}}, "
        + f"Wahrheit: \\texttt{{{class_truth}}} "
        + f"({os.path.basename(filenames[i])})"
    )

    if y_val_predicted[i] != y_val[i]:
        img_bgr = cv2.imread(filenames[i])
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
        img_segmented = variants[variant]["apply_segmentation"](img_gray)
        img_segmented_rgb = cv2.cvtColor(img_segmented, cv2.COLOR_GRAY2RGB)
        cnt, _ = variants[variant]["exctact_features"](img_segmented)
        cv2.drawContours(img_rgb, cnt, -1, (255, 0, 0), 3)
        cv2.drawContours(img_segmented_rgb, cnt, -1, (255, 0, 0), 3)
        fig = plt.figure(figsize=(5.7, 2.25), layout="tight")
        ax = fig.add_subplot(1, 3, 1)
        ax.imshow(img_gray, cmap="Greys_r")
        ax.set_title("Grauwerte")
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax = fig.add_subplot(1, 3, 2)
        ax.imshow(img_rgb)
        ax.set_title("Grauw. mit Kontur")
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        ax = fig.add_subplot(1, 3, 3)
        ax.imshow(img_segmented_rgb)
        ax.set_title("Segm. mit Kontur")
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        fig.suptitle(comment)

        if EXPORT_REPORT:
            fig.savefig(
                f"report/images/error_analysis_var{variant}_"
                + f"{os.path.splitext(os.path.basename(filenames[i]))[0]}.eps",
                pad_inches=0,
            )

## 9 Hyperparameter Tuning

Optimieren Sie Ihr Ergebnis und stellen Sie die Accuracy von mindestens 2 Varianten der Segmentierung und 2 Varianten von Merkmalskombinationen gegenüber $\Rightarrow$ 4 Ergebnisse
